# MRJOB 

In [1]:
#instalacion de mrjob
!pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 439 kB 5.2 MB/s 


In [2]:
#importacion de librerias
import numpy as np

In [5]:
#Creamos un archivo csv, una matriz
array_number = np.random.uniform(1, 100, size = (5, 15))
np.savetxt('arreglo_numerico.csv', array_number, delimiter = ',')
array_number.shape

(5, 15)

In [6]:
#Estadisticos con numpy
print('La suma de los valores es: ',np.sum(array_number))
print('Es un arreglo de tamaño: ', len(array_number.flatten()))
print('La media de array_number es:',np.mean(array_number))

La suma de los valores es:  3466.0601299394266
Es un arreglo de tamaño:  75
La media de array_number es: 46.21413506585902


In [ ]:
1+3+4+5+6+7+8+9+5+3+1+3+4+5+6

In [7]:
%%file mean_example.py
#Guardamos el programa mrjob en mean_example.py

from mrjob.job import MRJob

#Declaramos una clase de tipo MRJOB
class MeanMR(MRJob):
  #Declaramos el mapper 
  def mapper(self, _, line):
    valores = [float(value) for value in line.split(',')]
    longitud = len(valores) 
    yield (None , [sum(valores),longitud] ) 

  #Declaramos reducer
  def reducer(self, key, suma_de_valores):
    numerador = 0
    denominador = 0
    for i,j in suma_de_valores:
      numerador+=i
      denominador+=j
    yield key , numerador/denominador

if __name__ == '__main__':
  MeanMR.run()

Writing mean_example.py


In [8]:
!python mean_example.py arreglo_numerico.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/mean_example.root.20220910.232732.021600
Running step 1 of 1...
job output is in /tmp/mean_example.root.20220910.232732.021600/output
Streaming final output from /tmp/mean_example.root.20220910.232732.021600/output...
null	46.21413506585902
Removing temp directory /tmp/mean_example.root.20220910.232732.021600...


In [9]:
#Otra Manera de crear un archivo separado por comas
%%file numbers.txt
1, 3 ,4 , 5 ,6 
4,5,6
7
7,8,9,0,2,3

Writing numbers.txt


In [10]:
np.mean([1, 3 ,4 , 5 ,6 ,4,5,6,7,7,8,9,0,2,3])

4.666666666666667

In [11]:
!python mean_example.py numbers.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/mean_example.root.20220910.232924.644015
Running step 1 of 1...
job output is in /tmp/mean_example.root.20220910.232924.644015/output
Streaming final output from /tmp/mean_example.root.20220910.232924.644015/output...
null	4.666666666666667
Removing temp directory /tmp/mean_example.root.20220910.232924.644015...


In [ ]:
# Otro ejemplo utilizando mrjob

In [12]:
%%file fruits.txt
apple,banana,avocato,apple,apple,banana,avocato,apple,apple,banana,avocato,apple
apple,banana,avocato,apple
apple,banana,avocato,apple,apple,banana,avocato,apple

Writing fruits.txt


In [13]:
%%file fruits_example.py
from mrjob.job import MRJob
class FreqMR(MRJob):
    def mapper(self, _, line):
        for fruit in line.split(","):
            if (fruit=="apple"): 
              yield "apple", 1
            if (fruit=="banana"): 
              yield "banana", 1
            if (fruit=="avocato"): 
              yield "avocato", 1
        
    def reducer(self, key, values):          
        yield key, sum(values)
        
if __name__ == '__main__':
    FreqMR.run()

Writing fruits_example.py


In [14]:
!python fruits_example.py fruits.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/fruits_example.root.20220910.233440.337207
Running step 1 of 1...
job output is in /tmp/fruits_example.root.20220910.233440.337207/output
Streaming final output from /tmp/fruits_example.root.20220910.233440.337207/output...
"apple"	12
"avocato"	6
"banana"	6
Removing temp directory /tmp/fruits_example.root.20220910.233440.337207...


# pyspark

In [15]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop3.2.tgz
!tar xf spark-3.1.3-bin-hadoop3.2.tgz
!pip install -q findspark

In [16]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop3.2"

In [17]:
#Pyspark
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext

In [18]:
sc = SparkContext()

In [23]:
rdd = sc.parallelize([1,2,3,4,5,5,5,4,3,2,1])

In [24]:
rdd.collect()

[1, 2, 3, 4, 5, 5, 5, 4, 3, 2, 1]

In [26]:
from sklearn.datasets import load_boston
dataset = load_boston()
X = dataset['data']
y = dataset['target']

In [27]:
X_rdd = sc.parallelize(X.tolist())

In [41]:
#Verificando el funcionamiento del siguiente bloque de codigo
"""
tmp_val = X_rdd.flatMap(lambda sample: [(col, sample[col]) for col in range(len(sample))])
max_val = tmp_val.reduceByKey(lambda x, y: x if x > y else y).sortByKey().collect()
n_max = [val[1] for val in max_val]
datos_rdd.map(lambda sample: np.divide(np.subtract(sample, n_min), np.subtract(n_max, n_min)).tolist())
"""

'\ntmp_val = X_rdd.flatMap(lambda sample: [(col, sample[col]) for col in range(len(sample))])\nmax_val = tmp_val.reduceByKey(lambda x, y: x if x > y else y).sortByKey().collect()\nn_max = [val[1] for val in max_val]\ndatos_rdd.map(lambda sample: np.divide(np.subtract(sample, n_min), np.subtract(n_max, n_min)).tolist())\n'

In [39]:
#Escalar minmax
import numpy as np
def minMaxScaler(datos_rdd):
  tmp_val = datos_rdd.flatMap(lambda sample: [(col, sample[col]) for col in range(len(sample))])
  max_val = tmp_val.reduceByKey(lambda x, y: x if x > y else y).sortByKey().collect()
  min_val = tmp_val.reduceByKey(lambda x, y: x if x < y else y).sortByKey().collect()
  #Desechar la llave que acompaña a los valores maximos y minimos
  n_max = [val[1] for val in max_val]
  n_min = [val[1] for val in min_val]
  return datos_rdd.map(lambda sample: np.divide(np.subtract(sample, n_min), np.subtract(n_max, n_min)).tolist())
minMaxScaler(X_rdd).collect()

[[0.0,
  0.18,
  0.06781524926686218,
  0.0,
  0.3148148148148149,
  0.5775052692086607,
  0.6416065911431514,
  0.26920313906646415,
  0.0,
  0.20801526717557253,
  0.28723404255319157,
  1.0,
  0.08967991169977926],
 [0.00023592253917842758,
  0.0,
  0.2423020527859238,
  0.0,
  0.17283950617283944,
  0.5479977007089482,
  0.7826982492276005,
  0.34896198019441843,
  0.043478260869565216,
  0.1049618320610687,
  0.5531914893617023,
  1.0,
  0.20447019867549668],
 [0.00023569774400055386,
  0.0,
  0.2423020527859238,
  0.0,
  0.17283950617283944,
  0.6943858976815482,
  0.5993820803295572,
  0.34896198019441843,
  0.043478260869565216,
  0.1049618320610687,
  0.5531914893617023,
  0.9897372535175752,
  0.06346578366445917],
 [0.00029279571918046875,
  0.0,
  0.06304985337243403,
  0.0,
  0.15020576131687244,
  0.6585552787890402,
  0.44181256436663235,
  0.4485445898389546,
  0.08695652173913043,
  0.06679389312977099,
  0.648936170212766,
  0.9942760603156993,
  0.033388520971302425]

In [40]:
np.array(minMaxScaler(X_rdd).collect())

array([[0.00000000e+00, 1.80000000e-01, 6.78152493e-02, ...,
        2.87234043e-01, 1.00000000e+00, 8.96799117e-02],
       [2.35922539e-04, 0.00000000e+00, 2.42302053e-01, ...,
        5.53191489e-01, 1.00000000e+00, 2.04470199e-01],
       [2.35697744e-04, 0.00000000e+00, 2.42302053e-01, ...,
        5.53191489e-01, 9.89737254e-01, 6.34657837e-02],
       ...,
       [6.11892474e-04, 0.00000000e+00, 4.20454545e-01, ...,
        8.93617021e-01, 1.00000000e+00, 1.07891832e-01],
       [1.16072990e-03, 0.00000000e+00, 4.20454545e-01, ...,
        8.93617021e-01, 9.91300620e-01, 1.31070640e-01],
       [4.61841693e-04, 0.00000000e+00, 4.20454545e-01, ...,
        8.93617021e-01, 1.00000000e+00, 1.69701987e-01]])

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
X = dataset['data']
y = dataset['target']

In [ ]:
dataset_rdd = sc.parallelize(np.append(X, y[:, None], axis = 1), 5)

# Un poco de naive bayes

https://scikit-learn.org/stable/modules/naive_bayes.html


In [31]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [52]:
#numero de clases
print(gnb.class_count_)
#probabilidad apriori
print(gnb.class_prior_)
print(gnb.theta_)
#print(gnb.var_)

[29. 20. 26.]
[0.38666667 0.26666667 0.34666667]
[[4.97586207 3.35862069 1.44827586 0.23448276]
 [5.935      2.71       4.185      1.3       ]
 [6.77692308 3.09230769 5.73461538 2.10769231]]


In [36]:
print(29/np.sum(gnb.class_count_))
print(20/np.sum(gnb.class_count_))
print(26/np.sum(gnb.class_count_))

0.38666666666666666
0.26666666666666666
0.3466666666666667
